# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [21]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns



In [30]:
# DESCARGAR HTML

url = ' https://ycharts.com/companies/TSLA/revenues'
response = requests.get(url).text

# Si se recibe un "403 Forbidden", reintentamos con el User-Agent adecuado
if '403 Forbidden' in response:
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}
    request = requests.get(url, headers = headers)
    time.sleep(10)
    response = request.text 
html_data = response 

In [50]:
# TRANSFORMAR EL HTML 

soup = BeautifulSoup(html_data,"html.parser")
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>ERROR: The request could not be satisfied</title>
</head><body>
<h1>403 ERROR</h1>
<h2>The request could not be satisfied.</h2>
<hr noshade="" size="1px"/>
Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<br clear="all"/>
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<br clear="all"/>
<hr noshade="" size="1px"/>
<pre>
Generated by cloudfront (CloudFront)
Request ID: CHxQPymoyflAKADONqaZIl9yrzKnohrwvDW3-sGDrM1ravHyMz9BBg==
</pre>
<address>
</address>
</body></html>

In [25]:
# Buscar todas las tablas
tables = soup.find_all("table")
tables

[]

In [47]:
table_index = None  # Inicializa table_index como None

for index, table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
        break

# Verifica si se encontró la tabla
if table_index is not None:
    print("Índice de la tabla que contiene 'Tesla Quarterly Revenue':", table_index)
else:
    print("No se encontró ninguna tabla que contenga 'Tesla Quarterly Revenue'.")

No se encontró ninguna tabla que contenga 'Tesla Quarterly Revenue'.


In [48]:
# Crear un DataFrame
tesla_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({
            "Date": Date,
            "Revenue": Revenue
        }, index = [0])], ignore_index = True)

tesla_revenue.head()

TypeError: list indices must be integers or slices, not NoneType

In [35]:
# Procesar el dataframe

tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue.head()

,Date,Revenue


In [36]:
connection = sqlite3.connect("Tesla.db")
connection

In [37]:
cursor = connection.cursor()
cursor.execute("""CREATE TABLE revenue (Date, Revenue)""")

In [39]:
tesla_tuplas = list(tesla_revenue.to_records(index = False))
tesla_tuplas[:5]

[]

In [40]:
cursor.executemany("INSERT INTO revenue VALUES (?,?)", tesla_tuples)
connection.commit()